# TODO: descrizione strumenti usati

In [11]:
# 1. Import e configurazioni
import pandas as pd

In [12]:
# 2. Caricamento dati e pulizia iniziale

# File locale
file_path = "Archivio_unico_indicatori_regionali.csv"

# Caricamento colonne
df = pd.read_csv(
    file_path,
    sep=';',
    low_memory=False,
    decimal=',',
    thousands='.'
)

print(f"File caricato: {file_path}")

# Visualizzo le prime 5 righe
df.head()

File caricato: Archivio_unico_indicatori_regionali.csv


,COD_INDICATORE,TITOLO,SOTTOTITOLO,ANNO_RIFERIMENTO,UNITA_MISURA,VALORE,ID_RIPARTIZIONE,DESCRIZIONE_RIPARTIZIONE,ID_ASSE,DESCRIZIONE_ASSE_QCS,...,DESCRIZIONE_PRIORITA_QSN,ID_TEMA1,DESCRIZIONE_TEMA1,ID_TEMA2,DESCRIZIONE_TEMA2,1° ID_OBIETTIVO,1° OBIETTIVO,2° ID_OBIETTIVO,2° OBIETTIVO,OC_TEMA_SINTETICO
0,1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1995,migliaia di euro,"21,7996899471312",1,Piemonte,4,Asse IV - Sistemi locali di sviluppo.,...,7 - Competitività dei sistemi produttivi e occ...,34,Dinamiche settoriali,NaN,NaN,3.0,Obiettivo Tematico 3,NaN,NaN,Competitività per le imprese
1,1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1996,migliaia di euro,"22,1301853488336",1,Piemonte,4,Asse IV - Sistemi locali di sviluppo.,...,7 - Competitività dei sistemi produttivi e occ...,34,Dinamiche settoriali,NaN,NaN,3.0,Obiettivo Tematico 3,NaN,NaN,Competitività per le imprese
2,1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1997,migliaia di euro,"21,8351440567581",1,Piemonte,4,Asse IV - Sistemi locali di sviluppo.,...,7 - Competitività dei sistemi produttivi e occ...,34,Dinamiche settoriali,NaN,NaN,3.0,Obiettivo Tematico 3,NaN,NaN,Competitività per le imprese
3,1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1998,migliaia di euro,"22,5576057099927",1,Piemonte,4,Asse IV - Sistemi locali di sviluppo.,...,7 - Competitività dei sistemi produttivi e occ...,34,Dinamiche settoriali,NaN,NaN,3.0,Obiettivo Tematico 3,NaN,NaN,Competitività per le imprese
4,1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1999,migliaia di euro,"24,2888539200584",1,Piemonte,4,Asse IV - Sistemi locali di sviluppo.,...,7 - Competitività dei sistemi produttivi e occ...,34,Dinamiche settoriali,NaN,NaN,3.0,Obiettivo Tematico 3,NaN,NaN,Competitività per le imprese


In [ ]:
# 3. Pulizia dei dati

df = df.copy()

# Normalizzazione e sistemazione delle colonne
df.columns = [
    c.strip()
     .replace(" ", "_")
     .replace("°", "o")
     .replace("'", "")
     .lower()
    for c in df.columns
]

df.rename(columns={
    'titolo': 'indicatore',
    'anno_riferimento': 'anno',
    'descrizione_ripartizione': 'regione',
}, inplace=True)

# Conversione del valore numerico
if 'valore' in df.columns:
    df['valore'] = (
        df['valore']
        .astype(str)
        .str.replace('\u00A0', '', regex=False)
        .str.replace(' ', '', regex=False)
    )

    df['valore'] = df['valore'].str.replace(',', '.', regex=False)
    df['valore'] = pd.to_numeric(df['valore'], errors='coerce')

# Tengo in considerazione solo le colonne utili
keep_cols = [
    'indicatore', 'sottotitolo', 'anno',
    'unita_misura', 'valore', 'regione', 'oc_tema_sintetico'
]
df_clean = df[keep_cols].copy()

# Creazione colonna con valore standardizzato
df_clean['valore_std'] = df_clean['valore']

# Conversione per unità di misura
mask_migliaia = df_clean['unita_misura'].str.contains('migliaia', case=False, na=False)
mask_milioni  = df_clean['unita_misura'].str.contains('milioni',  case=False, na=False)

df_clean.loc[mask_migliaia, 'valore_std'] = df_clean.loc[mask_migliaia, 'valore_std'] * 1000
df_clean.loc[mask_milioni,  'valore_std'] = df_clean.loc[mask_milioni,  'valore_std'] * 1_000_000

# Visualizzazione delle prime 5 righe
df_clean.head()


,indicatore,sottotitolo,anno,unita_misura,valore,regione,valore_std
0,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1995,migliaia di euro,21.799690,Piemonte,21799.689947
1,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1996,migliaia di euro,22.130185,Piemonte,22130.185349
2,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1997,migliaia di euro,21.835144,Piemonte,21835.144057
3,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1998,migliaia di euro,22.557606,Piemonte,22557.605710
4,Produttività del lavoro in agricoltura,"Valore aggiunto dell'agricoltura, della caccia...",1999,migliaia di euro,24.288854,Piemonte,24288.853920


In [14]:
# 4. Visualizzazione macrotemi

print("Diamo un occhiata ai macrotemi:")
print(df_clean['oc_tema_sintetico'].unique())

Diamo un occhiata ai macrotemi:


KeyError: 'oc_tema_sintetico'